## S2 To R1 Attractor, S2 Damping - Geometric Consistency Test (PBDS)

In [ ]:
using PBDS, StaticArrays, LinearAlgebra, BenchmarkTools

## Setup

### Point Distance Attractor

In [ ]:
PBDS.task_map_emb(::EmbRep, ::EmbRep, xme, task_map::DistanceFromPoint{S2,R1}) =
    SA[norm(xme - task_map.position_center)]
PBDS.domain_coord_rep(::DistanceFromPoint{S2,R1}) = EmbRep()

PBDS.metric_chart(xn, task::Attractor{<:DistanceFromPoint{S2,R1}}, CN::Chart{1,R1}) = 
    default_metric(xn, task, CN)
PBDS.potential_chart(xn, task::Attractor{<:DistanceFromPoint{S2,R1}}, CN::Chart{1,R1}) = xn[1]^2
PBDS.dissipative_forces_chart(xn, vn, task::Attractor{<:DistanceFromPoint{S2,R1}}, CN::Chart{1,R1}) =  0*vn
PBDS.weight_metric_chart(xn, vn, task::Attractor{<:DistanceFromPoint{S2,R1}}, CN::Chart{1,R1}) = 
    default_weight_metric(xn, vn, task, CN)

### Sphere Damping

In [ ]:
PBDS.default_coord_rep(::Damping{<:Identity{S2,S2}}) = EmbRep()
PBDS.metric_emb(xne, task::Damping{<:Identity{S2,S2}}) =
    default_metric(xne, task)
PBDS.potential_emb(xne, task::Damping{<:Identity{S2,S2}}) = 0.
PBDS.dissipative_forces_emb(xne, vne, task::Damping{<:Identity{S2,S2}}) = -4*vne
PBDS.weight_metric_emb(xne, vne, task::Damping{<:Identity{S2,S2}}) =
    default_weight_metric(xne, vne, task)
PBDS.home_task_chart(task::Damping{<:Identity{S2,S2}}) = Chart{SterProjSouth,S2}()

In [ ]:
M = S2
tasks, CNs = TaskList(), ChartList()

N = R1
CN = Chart{1,N}()
center = SA[1., -1., -1.]
center = center ./ norm(center)
push!(tasks, Attractor(DistanceFromPoint{M,N}(center)))
push!(CNs, CN)

N = S2
CN = Chart{SterProjNorth,S2}()
push!(tasks, Damping(Identity{M,N,Float64}()))
push!(CNs, CN)

## Point Acceleration

In [ ]:
# Initial state
xme = SA[-1., 1., 1.]
xme = xme ./ norm(xme)
vme = SA[2., 1., 1.]
robot_coord_rep = EmbRep()

CM = Chart{SterProjNorth,S2}()
PBDS.choose_chart_emb(::EmbRep, pe, ::Chart{<:SterProj,S2}) = Chart{SterProjNorth,S2}()
σxddot, = multiple_task_acceleration(xme, vme, tasks, CM, CNs, robot_coord_rep)

In [ ]:
CM = Chart{SterProjSouth,S2}()
PBDS.choose_chart_emb(::EmbRep, pe, ::Chart{<:SterProj,S2}) = Chart{SterProjSouth,S2}()
σxddot, = multiple_task_acceleration(xme, vme, tasks, CM, CNs, robot_coord_rep)

## Single Trajectory

In [ ]:
using Plots, Makie, Observables, ProgressMeter

In [ ]:
Time = 15
dt = 0.01

PBDS.choose_chart_emb(::EmbRep, pe, ::Chart{<:SterProj,S2}) = Chart{SterProjNorth,S2}()
traj_north = propagate_tasks(xme, vme, tasks, CM, CNs, Time, dt, robot_coord_rep, log_tasks = true)

PBDS.choose_chart_emb(::EmbRep, pe, ::Chart{<:SterProj,S2}) = Chart{SterProjSouth,S2}()
traj_south = propagate_tasks(xme, vme, tasks, CM, CNs, Time, dt, robot_coord_rep, log_tasks = true)

PBDS.choose_chart_emb(::EmbRep, pe, ::Chart{<:SterProj,S2}) =
    (pe[3] < 0)[1] ? Chart{SterProjSouth,S2}() : Chart{SterProjNorth,S2}()
traj_switching = propagate_tasks(xme, vme, tasks, CM, CNs, Time, dt, robot_coord_rep, log_tasks = true)
traj_switching.xm[end]

In [ ]:
Nplot = length(traj_switching.xm)
Plots.plot(getindex.(traj_switching.xm,1)[1:Nplot])
Plots.plot!(getindex.(traj_switching.xm,2)[1:Nplot])
Plots.plot!(getindex.(traj_switching.xm,3)[1:Nplot])

In [ ]:
ax_size, plot_size = 1, 800
limits = FRect3D((-ax_size, -ax_size, -ax_size), (2*ax_size, 2*ax_size, 2*ax_size))
scene = Scene(resolution = (plot_size, plot_size))
mesh!(Sphere(Point3(zeros(3)), 1.), color = RGBA(1.,1.,1.,0.4), transparency = true)
Makie.scatter!(scene, [xme[1]], [xme[2]], [xme[3]], markersize = ax_size/20, color = :blue, limits = limits)
δ = 0.99
Makie.scatter!(scene, [center[1]*δ], [center[2]*δ], [center[3]*δ], markersize = ax_size/20, color = :green, limits = limits)

linewidth = 1.5
Makie.lines!(scene, getindex.(traj_switching.xm,1), getindex.(traj_switching.xm,2), getindex.(traj_switching.xm,3), color = :purple, linewidth = linewidth)
Makie.lines!(scene, getindex.(traj_south.xm,1), getindex.(traj_south.xm,2), getindex.(traj_south.xm,3), color = :red, linewidth = linewidth)
Makie.lines!(scene, getindex.(traj_north.xm,1), getindex.(traj_north.xm,2), getindex.(traj_north.xm,3), color = :blue, linewidth = linewidth)

axis = scene[Axis]
axis.showaxis = false
rotate_cam!(scene, 0.4, 0., 0.)
display(scene)

## Animation

In [ ]:
iobs = Observable(1)
AbstractPlotting.__init__()
Makie.AbstractPlotting.inline!(false)
ax_size, plot_size = 1, 800
limits = FRect3D((-ax_size, -ax_size, -ax_size), (2*ax_size, 2*ax_size, 2*ax_size))
scene = Scene(resolution = (plot_size, plot_size))
mesh!(Sphere(Point3f0(zeros(3)), 1.), color = RGBA(1.,1.,1.,0.4), transparency = true)
δ = 1
Makie.scatter!(scene, [center[1]*δ], [center[2]*δ], [center[3]*δ], markersize = ax_size/20, color = :green, limits = limits)

accel_rate = 2
# Makie.scatter!(scene, lift(i -> [traj_switching.xm[(i-1)*accel_rate+1][1]], iobs), lift(i -> [traj_switching.xm[(i-1)*accel_rate+1][2]], iobs), lift(i -> [traj_switching.xm[(i-1)*accel_rate+1][3]], iobs), markersize = ax_size/16, color = :purple, limits = limits)
# Makie.scatter!(scene, lift(i -> [traj_south.xm[(i-1)*accel_rate+1][1]], iobs), lift(i -> [traj_south.xm[(i-1)*accel_rate+1][2]], iobs), lift(i -> [traj_south.xm[(i-1)*accel_rate+1][3]], iobs), markersize = ax_size/18, color = :red, limits = limits)
Makie.scatter!(scene, lift(i -> [traj_north.xm[(i-1)*accel_rate+1][1]], iobs), lift(i -> [traj_north.xm[(i-1)*accel_rate+1][2]], iobs), lift(i -> [traj_north.xm[(i-1)*accel_rate+1][3]], iobs), markersize = ax_size/20, color = :blue, limits = limits)

linewidth = 2.5
Makie.lines!(scene, getindex.(traj_switching.xm,1), getindex.(traj_switching.xm,2), getindex.(traj_switching.xm,3), color = :purple, linewidth = linewidth)
Makie.lines!(scene, getindex.(traj_south.xm,1), getindex.(traj_south.xm,2), getindex.(traj_south.xm,3), color = :red, linewidth = linewidth)
Makie.lines!(scene, getindex.(traj_north.xm,1), getindex.(traj_north.xm,2), getindex.(traj_north.xm,3), color = :blue, linewidth = linewidth)

Makie.xlabel!(scene, "x")
Makie.ylabel!(scene, "y")
axis = scene[Axis]
axis.showaxis = false
rotate_cam!(scene, 0.4, 0., 0.)
display(scene)

In [ ]:
function record_scene(scene, filename, iobs, N, framerate=60)
    p = Progress(N)
    record(scene, filename, 1:N) do i
        iobs[] = i
        rotate_cam!(scene, 0.01, 0., 0.)
        isdefined(Main, :Test) || next!(p)
    end
    isdefined(Main, :Test) || display("text/html", html_video(filename))
end

filename = "S2_To_R1Attractor_S2Damping_ConsistencyTest.mp4"
record_scene(scene, filename, iobs, Int(floor(length(traj_north.xm)/accel_rate)))